In [1]:
# import time libraries
import time
from datetime import datetime
from time import gmtime, strftime

# Set Begin Time to Measure Time to Run of Whole Notebook
begin_time = time.time()

In [2]:
## Pull Relavent Libraries

# Standard Libraries
import pandas as pd
import numpy as np
import os
import math
import requests
from io import StringIO
#from StringIO import StringIO
import sys
import os.path
from IPython.display import display, HTML

# For Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Other
import scipy
from scipy.stats import norm, skew #for some statistics
from scipy.special import boxcox1p
from datetime import timedelta

# For Confusion Matrix
from sklearn.metrics import confusion_matrix
import itertools

# Import Functions Built
from Functions import countplotshorizontal, null_values, histogram, stacked_bar, anova, spearman_df, spearman_chart, encode, QQ, plot_confusion_matrix

# Set Notebook Options
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
bold = "\033[1m"
reset = "\033[0;0m"

## Modeling Libraries
## Create _csv.reader object
import csv
%pylab inline

from itertools import combinations
from numpy import array,array_equal

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn import tree
from sklearn import metrics
from sklearn import ensemble
from sklearn import linear_model 
from sklearn import naive_bayes 

import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import xgboost as feature_names

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score

Populating the interactive namespace from numpy and matplotlib


C:\Users\Armand\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['histogram', 'norm', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
# Read in the DataFrames
os.chdir(sys.path[0] + '\\Data')
# os.chdir('C:\\Users\\apost\\Desktop\\Python\\CL\\LendingClub\\Data')

df2 = pd.read_pickle('df2.pkl')
df3 = pd.read_pickle('df3.pkl')

df_test = pd.read_pickle('df_test.pkl')
df_OOT = pd.read_pickle('df_OOT.pkl')

df_test_XGB = pd.read_pickle('df_test_XGB.pkl')
df_OOT_XGB = pd.read_pickle('df_OOT_XGB.pkl')

display(HTML(df_test[0:5].to_html()))

,target,id,roa,int_rate,grade_A,grade_B,grade_C,grade_D,grade_E,OOT_Ind,funded_amnt,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,time_in_file,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,bc_open_to_buy,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_tl,num_il_tl,num_rev_accts,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,fico_avg,term__36_months,term__60_months,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_Other,home_ownership_RENT,verification_status_Not_Verified,verification_status_Source_Verified,verification_status_Verified,purpose_Other,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,addr_state_CA,addr_state_FL,addr_state_NY,addr_state_Other,addr_state_TX,initial_list_status_f,initial_list_status_w,target_actual,target_pred
0,0,7675953,4,9,1,0,0,0,0,0,20000,3,108,252,0,0,32,0,224,487,31,59,0,34,257,4,13290,158,372,20,20,1,5,24,1,6,7,16,9,21,0,2,73,42,0,265,247,252,7070,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0
1,0,35793395,-69,13,0,0,1,0,0,0,12000,10,110,318,0,0,31,0,236,753,16,52,0,36,249,6,7138,128,193,20,20,1,5,10,1,5,7,7,4,9,0,3,100,60,0,296,233,294,6870,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0
2,0,6695915,9,15,0,0,1,0,0,0,4000,0,121,66,0,2,18,0,227,875,18,54,0,25,233,2,2836,225,227,34,34,2,15,1,1,2,2,5,10,6,0,0,83,100,0,258,233,263,6870,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0
3,1,9229456,14,19,0,0,0,1,0,0,8000,2,113,135,0,1,26,0,177,348,9,41,0,34,219,7,1063,7,65,7,7,1,24,1,0,2,2,2,2,6,0,4,100,100,0,273,183,272,7020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1
4,0,6158826,4,6,1,0,0,0,0,0,12000,1,114,126,0,0,31,0,205,236,30,52,0,39,271,7,33583,166,191,18,15,1,4,10,0,5,6,8,18,11,0,4,100,0,0,300,265,292,7770,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0


In [4]:
df2_35793395 = df2.loc[df2['id'] == 35793395]
df3_35793395 = df3.loc[df3['id'] == 35793395]
df_test_35793395 = df_test.loc[df_test['id'] == 35793395]
# df_34923386 = df.loc[df['id'] == 34923386]

display(HTML(df2_35793395.to_html()))
display(HTML(df3_35793395.to_html()))
display(HTML(df_test_35793395.to_html()))

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,count,YearMonth,YearQtr,fico_avg,fico_bin,GCL,NCL,replacement_d,time_in_file,term_int,loan_end_date,life_of_loan,life_of_loan_years,servicing_fee,total_rev,annual_rev,roa,term_E,grade_E,sub_grade_E,home_ownership_E,verification_status_E,purpose_E,addr_state_E,initial_list_status_E,debt_settlement_flag_E
14285,35793395,12000,12000.000,60 months,13.660,277.110,C,C3,10,MORTGAGE,60000,Source Verified,2014-11-01,Charged Off,n,debt_consolidation,394xx,MS,31.800,0,1998-10-01,685,689,0,12,0,24258.000,75.300,16,w,3895.400,3895.400,1017.800,912.860,0.000,1964.740,176.827,2015-06-01,277.110,2016-10-01,644,640,0,1,Individual,0,0,254815,32200,6,21235,7138,68.300,0,0,128,193,5,5,1,5,10,1,5,7,5,7,4,7,9,7,12,0,0,0,3,100.000,60.000,0,0,286621,81461,22500,78021,N,Cash,N,1,201411,201404,687.000,682 to 697,-8104.600,-9892.513,2014-12-01,193.018,60,2019-11-01,12.000,1.000,38.954,-8320.381,-8320.381,-69.337,1.000,3.000,23.000,4.000,1.000,5.000,2.000,1.000,2.000


,id,issue_d,OOT_Ind,funded_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,time_in_file,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,bc_open_to_buy,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_tl,num_il_tl,num_rev_accts,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,fico_avg,roa,term__36_months,term__60_months,grade_A,grade_B,grade_C,grade_D,grade_E,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_Other,home_ownership_RENT,verification_status_Not_Verified,verification_status_Source_Verified,verification_status_Verified,purpose_Other,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,addr_state_CA,addr_state_FL,addr_state_NY,addr_state_Other,addr_state_TX,initial_list_status_f,initial_list_status_w
147395,35793395,2014-11-01,0,12000.000,13.660,10,11.002,31.800,0,0,3.128,0,23.647,75.300,16,5.263,0,36.470,24.963,6,7138,128,193,2.056,2.056,1,5,10,1,5,7,7,4,9,0.000,3,100.000,60.000,0,29.687,23.307,29.453,687.000,-69.337,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1


,target,id,roa,int_rate,grade_A,grade_B,grade_C,grade_D,grade_E,OOT_Ind,funded_amnt,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,time_in_file,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,bc_open_to_buy,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_tl,num_il_tl,num_rev_accts,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,fico_avg,term__36_months,term__60_months,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_Other,home_ownership_RENT,verification_status_Not_Verified,verification_status_Source_Verified,verification_status_Verified,purpose_Other,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,addr_state_CA,addr_state_FL,addr_state_NY,addr_state_Other,addr_state_TX,initial_list_status_f,initial_list_status_w,target_actual,target_pred
1,0,35793395,-69,13,0,0,1,0,0,0,12000,10,110,318,0,0,31,0,236,753,16,52,0,36,249,6,7138,128,193,20,20,1,5,10,1,5,7,7,4,9,0,3,100,60,0,296,233,294,6870,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0


In [5]:
df = df2[['id','roa','issue_d','loan_amnt','life_of_loan_years','int_rate','fico_avg','grade','sub_grade','emp_length','home_ownership','annual_inc','dti','revol_util','acc_open_past_24mths','num_accts_ever_120_pd','purpose']].copy(deep = True)
df_test_pred = df_test[['id','target','target_pred']].copy(deep = True)
df_OOT_pred = df_OOT[['id','target','target_pred']].copy(deep = True)

In [6]:
df_comb = pd.merge(df_test_pred, df, how = 'left', left_on = 'id', right_on = 'id', copy = False)
df_comb['roa_num'] = (df_comb['roa'] * df_comb['loan_amnt'])

display(HTML(df_comb[0:5].to_html()))

,id,target,target_pred,roa,issue_d,loan_amnt,life_of_loan_years,int_rate,fico_avg,grade,sub_grade,emp_length,home_ownership,annual_inc,dti,revol_util,acc_open_past_24mths,num_accts_ever_120_pd,purpose,roa_num
0,7675953,0,0,4.582,2013-10-01,20000,3.001,9.250,707.000,A,A5,3,MORTGAGE,52000,25.250,48.700,4,4,debt_consolidation,91640.656
1,35793395,0,0,-69.337,2014-11-01,12000,1.000,13.660,687.000,C,C3,10,MORTGAGE,60000,31.800,75.300,6,1,debt_consolidation,-832038.060
2,6695915,0,0,9.709,2013-08-01,4000,2.502,15.880,687.000,C,C4,0,RENT,180000,6.690,87.500,2,1,major_purchase,38835.174
3,9229456,1,1,14.225,2013-12-01,8000,1.000,19.220,702.000,D,D4,2,MORTGAGE,85000,13.570,34.800,7,0,debt_consolidation,113803.625
4,6158826,0,0,4.029,2013-07-01,12000,1.665,6.030,777.000,A,A1,1,MORTGAGE,95000,12.670,23.600,7,0,debt_consolidation,48342.129


In [7]:
#df_comb.to_csv('df_comb.csv',index = True,header = True, sep = ',')

# GradePvt = pivot_table(df_comb, values = 'roa_num')

In [8]:
df_comb_OOT = pd.merge(df_OOT_pred, df, how = 'left', left_on = 'id', right_on = 'id', copy = False)
df_comb_OOT['roa_num'] = (df_comb_OOT['roa'] * df_comb['loan_amnt'])

display(HTML(df_comb_OOT[0:5].to_html()))

,id,target,target_pred,roa,issue_d,loan_amnt,life_of_loan_years,int_rate,fico_avg,grade,sub_grade,emp_length,home_ownership,annual_inc,dti,revol_util,acc_open_past_24mths,num_accts_ever_120_pd,purpose,roa_num
0,44429213,1,1,10.930,2015-03-01,10000,3.504,15.610,707.000,D,D1,10,RENT,30000,32.960,43.800,6,0,debt_consolidation,218608.207
1,44057008,0,0,7.930,2015-03-01,18000,1.840,9.990,752.000,B,B3,10,MORTGAGE,64709,34.330,64.800,7,0,car,95155.959
2,44299188,0,0,6.059,2015-03-01,24000,2.420,8.180,712.000,B,B1,10,MORTGAGE,82500,31.030,41.300,6,0,debt_consolidation,24237.562
3,44017917,0,0,2.210,2015-03-01,24000,1.000,5.930,762.000,A,A1,8,RENT,110480,16.030,26.200,7,0,debt_consolidation,17682.250
4,44259158,0,0,9.681,2015-03-01,35000,1.254,12.690,692.000,C,C2,10,MORTGAGE,86000,32.490,49.900,10,0,debt_consolidation,116170.778


In [9]:
df_comb['roa'].quantile([.1, .2, .4, .5, .6, .8, .9])

0.100   -17.737
0.200     2.988
0.400     5.741
0.500     6.765
0.600     7.847
0.800    10.497
0.900    12.754
Name: roa, dtype: float64

In [10]:
df_comb[['target_pred', 'grade', 'roa']].groupby(['target_pred','grade']).quantile([.1, .2, .4, .5, .6, .8, .9])

roa
target_pred grade              
0           A     0.100   2.689
                  0.200   3.133
                  0.400   3.749
                  0.500   3.983
                  0.600   4.271
                  0.800   4.971
                  0.900   5.528
            B     0.100  -0.011
                  0.200   4.815
                  0.400   5.937
                  0.500   6.331
                  0.600   6.752
                  0.800   7.958
                  0.900   8.874
            C     0.100 -19.700
                  0.200   2.995
                  0.400   7.551
                  0.500   8.202
                  0.600   8.853
                  0.800  10.507
                  0.900  11.538
            D     0.100 -39.640
                  0.200  -1.910
                  0.400   8.704
                  0.500   9.352
                  0.600   9.880
                  0.800  11.828
                  0.900  13.103
            E     0.100 -60.892
                  0.200 -36.784
                  0.400   8.420
                  0.500  12.362
                  0.600  13.516
                  0.800  16.955
                  0.900  19.250
1           C     0.100 -19.719
                  0.200   1.532
                  0.400   9.662
                  0.500  10.461
                  0.600  11.284
                  0.800  12.688
                  0.900  13.646
            D     0.100 -40.111
                  0.200  -7.053
                  0.400   9.891
                  0.500  10.796
                  0.600  11.891
                  0.800  13.949
                  0.900  15.067
            E     0.100 -58.022
                  0.200 -20.501
                  0.400   8.187
                  0.500  11.721
                  0.600  13.483
                  0.800  16.770
                  0.900  18.696

In [11]:
df_comb[['target_pred', 'roa']].groupby('target_pred').quantile([.1, .2, .4, .5, .6, .8, .9])

roa
target_pred              
0           0.100  -8.256
            0.200   3.417
            0.400   5.553
            0.500   6.435
            0.600   7.272
            0.800   9.181
            0.900  10.605
1           0.100 -48.121
            0.200 -11.472
            0.400   9.699
            0.500  11.035
            0.600  12.311
            0.800  14.888
            0.900  16.753

In [12]:
df_comb[['grade', 'roa']].groupby('grade').quantile([.1, .2, .4, .5, .6, .8, .9])

roa
grade              
A     0.100   2.689
      0.200   3.133
      0.400   3.749
      0.500   3.983
      0.600   4.271
      0.800   4.971
      0.900   5.528
B     0.100  -0.011
      0.200   4.815
      0.400   5.937
      0.500   6.331
      0.600   6.752
      0.800   7.958
      0.900   8.874
C     0.100 -19.701
      0.200   2.869
      0.400   7.601
      0.500   8.290
      0.600   9.016
      0.800  10.691
      0.900  11.769
D     0.100 -39.962
      0.200  -5.298
      0.400   9.100
      0.500   9.986
      0.600  10.930
      0.800  13.225
      0.900  14.408
E     0.100 -58.765
      0.200 -21.792
      0.400   8.221
      0.500  11.801
      0.600  13.489
      0.800  16.783
      0.900  18.811

In [13]:
df_comb[['roa']].quantile([.1, .2, .4, .5, .6, .8, .9])      

,roa
0.100,-17.737
0.200,2.988
0.400,5.741
0.500,6.765
0.600,7.847
0.800,10.497
0.900,12.754


In [14]:
df_comb_OOT[['roa']].quantile([.1, .2, .4, .5, .6, .8, .9])

,roa
0.100,-29.700
0.200,0.529
0.400,4.499
0.500,5.565
0.600,6.562
0.800,9.148
0.900,11.292


In [15]:
df_comb2 = df_comb.loc[df_comb['grade'] != 'D']
df_comb2 = df_comb2.loc[df_comb['grade'] != 'E']

df_roa_num = df_comb2[['target_pred', 'grade', 'roa_num']].groupby(['target_pred','grade']).sum()
df_roa_num = df_roa_num.reset_index(level = 'target_pred', col_level = 0)
df_roa_num = df_roa_num.reset_index(drop = False)

df_loan_amnt = df_comb2[['target_pred', 'grade', 'loan_amnt']].groupby(['target_pred','grade']).sum()
df_loan_amnt = df_loan_amnt.reset_index(level = 'target_pred', col_level = 0)
df_loan_amnt = df_loan_amnt.reset_index(drop = False)

df_comb_summ = pd.merge(df_roa_num, df_loan_amnt, how = 'left', left_on = None, right_on = None, copy = False)
df_comb_summ['roa'] = df_comb_summ['roa_num'] / df_comb_summ['loan_amnt']

display(HTML(df_comb_summ[['grade','target_pred','roa']].to_html()))

,grade,target_pred,roa
0,A,0,2.398
1,B,0,2.784
2,C,0,1.500
3,C,1,3.103


In [16]:
df_comb2 = df_comb.loc[df_comb['grade'] != 'D']
df_comb2 = df_comb2.loc[df_comb2['grade'] != 'E']

df_roa_num = df_comb2[['target_pred', 'roa_num']].groupby(['target_pred']).sum()
df_roa_num = df_roa_num.reset_index(level = 'target_pred', col_level = 0)
df_roa_num = df_roa_num.reset_index(drop = False)

df_loan_amnt = df_comb2[['target_pred','loan_amnt']].groupby(['target_pred']).sum()
df_loan_amnt = df_loan_amnt.reset_index(level = 'target_pred', col_level = 0)
df_loan_amnt = df_loan_amnt.reset_index(drop = False)

df_comb_summ = pd.merge(df_roa_num, df_loan_amnt, how = 'left', left_on = None, right_on = None, copy = False)
df_comb_summ['roa'] = df_comb_summ['roa_num'] / df_comb_summ['loan_amnt']

display(HTML(df_comb_summ[['target_pred','roa']].to_html()))

,target_pred,roa
0,0,2.223
1,1,3.103


In [17]:
df_roa_num = df_comb_OOT[['target_pred', 'grade', 'roa_num']].groupby(['target_pred','grade']).sum()
df_roa_num = df_roa_num.reset_index(level = 'target_pred', col_level = 0)
df_roa_num = df_roa_num.reset_index(drop = False)

df_loan_amnt = df_comb_OOT[['target_pred', 'grade', 'loan_amnt']].groupby(['target_pred','grade']).sum()
df_loan_amnt = df_loan_amnt.reset_index(level = 'target_pred', col_level = 0)
df_loan_amnt = df_loan_amnt.reset_index(drop = False)

df_comb_summ = pd.merge(df_roa_num, df_loan_amnt, how = 'left', left_on = None, right_on = None, copy = False)
df_comb_summ['roa'] = df_comb_summ['roa_num'] / df_comb_summ['loan_amnt']

display(HTML(df_comb_summ[['grade','target_pred','roa']].to_html()))

,grade,target_pred,roa
0,A,0,1.678
1,B,0,1.118
2,C,0,-1.114
3,D,0,-3.023
4,E,0,-9.271
5,C,1,-0.177
6,D,1,-4.231
7,E,1,-6.612


In [18]:
df_comb_OOT2 = df_comb_OOT.loc[df_comb_OOT['grade'] != 'D']
df_comb_OOT2 = df_comb_OOT2.loc[df_comb_OOT2['grade'] != 'E']

df_roa_num = df_comb_OOT2[['target_pred', 'grade', 'roa_num']].groupby(['target_pred','grade']).sum()
df_roa_num = df_roa_num.reset_index(level = 'target_pred', col_level = 0)
df_roa_num = df_roa_num.reset_index(drop = False)

df_loan_amnt = df_comb_OOT2[['target_pred', 'grade', 'loan_amnt']].groupby(['target_pred','grade']).sum()
df_loan_amnt = df_loan_amnt.reset_index(level = 'target_pred', col_level = 0)
df_loan_amnt = df_loan_amnt.reset_index(drop = False)

df_comb_summ = pd.merge(df_roa_num, df_loan_amnt, how = 'left', left_on = None, right_on = None, copy = False)
df_comb_summ['roa'] = df_comb_summ['roa_num'] / df_comb_summ['loan_amnt']

display(HTML(df_comb_summ[['grade','target_pred','roa']].to_html()))

,grade,target_pred,roa
0,A,0,1.678
1,B,0,1.118
2,C,0,-1.114
3,C,1,-0.177


In [19]:
df_comb_OOT2 = df_comb_OOT.loc[df_comb_OOT['grade'] != 'D']
df_comb_OOT2 = df_comb_OOT2.loc[df_comb_OOT2['grade'] != 'E']

df_roa_num = df_comb_OOT2[['target_pred', 'roa_num']].groupby(['target_pred']).sum()
df_roa_num = df_roa_num.reset_index(level = 'target_pred', col_level = 0)
df_roa_num = df_roa_num.reset_index(drop = False)

df_loan_amnt = df_comb_OOT2[['target_pred', 'loan_amnt']].groupby(['target_pred']).sum()
df_loan_amnt = df_loan_amnt.reset_index(level = 'target_pred', col_level = 0)
df_loan_amnt = df_loan_amnt.reset_index(drop = False)

df_comb_summ = pd.merge(df_roa_num, df_loan_amnt, how = 'left', left_on = None, right_on = None, copy = False)
df_comb_summ['roa'] = df_comb_summ['roa_num'] / df_comb_summ['loan_amnt']

display(HTML(df_comb_summ[['target_pred','roa']].to_html()))

,target_pred,roa
0,0,0.461
1,1,-0.177


In [20]:
end_time = time.time()

diff = end_time - begin_time

print("run time ...", "\n")
print(diff, "seconds")
print(diff / 60, "minutes")
print(diff / 3600, "hours")

run time ... 

2.6234662532806396 seconds
0.04372443755467733 minutes
0.0007287406259112888 hours
